# <span style="font-size:1.5em;"> Rain in Australia Data Preparation and Analysis
Authors: Angela Kim, Colin Pelzer, Daniel Burdeno, Steven Addison

---

# Contents
<l></l>

<span style="font-size:1.2em;">

- <a href="#Overview">Overview</a>

- <a href="#Business Understanding">Business Understanding</a>  

- <a href="#Data Understanding">Data Understanding</a>  

- <a href="#Data Preparation">Data Preparation</a>  
    
    - <a href="#Replacing Sunshine Null Values">Replacing Sunshine Null Values</a>
    
    - <a href="#Removing Cloud Cover Values Outside of Range">Removing Cloud Cover Values Outside of Range</a>

    - <a href="#Removing Outliers">Removing Outliers</a>  

    - <a href="#Preparing Data for Modeling">Preparing Data for Modeling: Imputing and Encoding</a>

- <a href="#Final Remarks">Final Remarks</a>

---

# <a id="#Overview">Overview</a>

> This project analyzes climate data of various locations in Australia and designs the best classification model to predict whether it will rain tomorrow based on different weather features such as relative humidity, cloud cover, and atmospheric pressure.

---

# <a id="#Business Understanding">Business Understanding</a>

<span style="font-size:1.1em;">

> Agriculture is an essential part of Australia's economy, and rainfall plays a big role in its success. Real-time weather forecasting can help farmers make better decisions on how to tend to their crops. It's important to know what kinds of weather conditions (ie. relative humidity, hours of sunshine) will determine precipitation in order to implement the best methods to protect crops and secure a high and robust yield.
    >
> For areas with low rainfall, it would be important to know if it will rain so that farmers know if they should prepare to irrigate their crops or not. For areas with high rainfall, farmers can use a predictive model to protect their crops from flooding. Another area of farming that depends on real-time rain forecasting is fertilizer application and timing. You want the appropriate amount of moisture in the soil so that the fertilizer can be worked in. Soil moisture also determines field workablity. Farmers can become more efficient in their operations and save costs on unnecessary irrigations and poor fertilization scheduling.

---

# <a id="Data Understanding">Data Understanding</a>

> First, we start with importing the relevant libraries to load and clean out our dataset.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer

In [2]:
# Read .csv file
df = pd.read_csv('data/weatherAUS.csv')

# First glance at dataframe
df.head(10)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
5,2008-12-06,Albury,14.6,29.7,0.2,NaN,NaN,WNW,56.0,W,...,55.0,23.0,1009.2,1005.4,NaN,NaN,20.6,28.9,No,No
6,2008-12-07,Albury,14.3,25.0,0.0,NaN,NaN,W,50.0,SW,...,49.0,19.0,1009.6,1008.2,1.0,NaN,18.1,24.6,No,No
7,2008-12-08,Albury,7.7,26.7,0.0,NaN,NaN,W,35.0,SSE,...,48.0,19.0,1013.4,1010.1,NaN,NaN,16.3,25.5,No,No
8,2008-12-09,Albury,9.7,31.9,0.0,NaN,NaN,NNW,80.0,SE,...,42.0,9.0,1008.9,1003.6,NaN,NaN,18.3,30.2,No,Yes
9,2008-12-10,Albury,13.1,30.1,1.4,NaN,NaN,W,28.0,S,...,58.0,27.0,1007.0,1005.7,NaN,NaN,20.1,28.2,Yes,No


> From a quick look at the first 10 rows, we can already see many NaN values, especially for `Evaporation` and `Sunshine`. All the columns related to `Wind Direction` and `Cloud Cover` are categorical. `Wind Direction`, `Wind Speed`, `Cloud Cover`, `Humidity`, and `Atmospheric Pressure` are recorded twice in a day: at 9am and 3pm.

In [3]:
# Count of non-null values, datatypes, total entries
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

> A quick glance at dataframe info shows a lot of missing values, 20 feature columns, and our target: `RainTomorrow`.

In [4]:
# Checking descriptive statistics
df.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
count,143975.000000,144199.000000,142199.000000,82670.000000,75625.000000,135197.000000,143693.000000,142398.000000,142806.000000,140953.000000,130395.00000,130432.000000,89572.000000,86102.000000,143693.000000,141851.00000
mean,12.194034,23.221348,2.360918,5.468232,7.611178,40.035230,14.043426,18.662657,68.880831,51.539116,1017.64994,1015.255889,4.447461,4.509930,16.990631,21.68339
std,6.398495,7.119049,8.478060,4.193704,3.785483,13.607062,8.915375,8.809800,19.029164,20.795902,7.10653,7.037414,2.887159,2.720357,6.488753,6.93665
min,-8.500000,-4.800000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,980.50000,977.100000,0.000000,0.000000,-7.200000,-5.40000
25%,7.600000,17.900000,0.000000,2.600000,4.800000,31.000000,7.000000,13.000000,57.000000,37.000000,1012.90000,1010.400000,1.000000,2.000000,12.300000,16.60000
50%,12.000000,22.600000,0.000000,4.800000,8.400000,39.000000,13.000000,19.000000,70.000000,52.000000,1017.60000,1015.200000,5.000000,5.000000,16.700000,21.10000
75%,16.900000,28.200000,0.800000,7.400000,10.600000,48.000000,19.000000,24.000000,83.000000,66.000000,1022.40000,1020.000000,7.000000,7.000000,21.600000,26.40000
max,33.900000,48.100000,371.000000,145.000000,14.500000,135.000000,130.000000,87.000000,100.000000,100.000000,1041.00000,1039.600000,9.000000,9.000000,40.200000,46.70000


In [5]:
# Check value counts
for c in df.columns:
    print ("---- %s ----" % c)
    print (df[c].value_counts())
    print("\n")

---- Date ----
2014-04-28    49
2015-12-02    49
2016-07-04    49
2015-04-01    49
2014-08-14    49
              ..
2007-12-13     1
2007-11-27     1
2007-12-30     1
2007-11-11     1
2007-11-22     1
Name: Date, Length: 3436, dtype: int64


---- Location ----
Canberra            3436
Sydney              3344
Perth               3193
Hobart              3193
Darwin              3193
Brisbane            3193
Melbourne           3193
Adelaide            3193
Albany              3040
MountGambier        3040
GoldCoast           3040
Launceston          3040
Ballarat            3040
Albury              3040
Wollongong          3040
AliceSprings        3040
Cairns              3040
Townsville          3040
MountGinini         3040
Bendigo             3040
Tuggeranong         3039
Newcastle           3039
Penrith             3039
Woomera             3009
BadgerysCreek       3009
CoffsHarbour        3009
WaggaWagga          3009
Mildura             3009
Richmond            3009
NorfolkIsland

In [6]:
# Count of null values per column & total
display(df.isna().sum())
print('Total Null Count:', df.isna().sum().sum())

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

Total Null Count: 343248


> After finding missing values in the first few rows of the data and checking how many missing values we are dealing with, we found that out of our 23 columns, only 2 of them had no missing values, `Date` and `Location`.
>
> We calculated the amount of missing values per feature column and total missing values from the dataset (343,248!!).
>
> Some columns such as `Evaporation`, `Sunshine`, `Cloud9am`, and `Cloud3pm` have over 40% null values.
>
> In order to understand these null values, we referred to the [description](https://rdrr.io/cran/rattle.data/man/weatherAUS.html) of the dataset and researched further to gain more domain knowledge. This led us to the Australian Government Bureau of Meteorology [(BOM)](http://www.bom.gov.au/climate/data/) site to fill in the missing values for `Sunshine`.
>
> We also took note of possible outliers, which we handle later.

---

# <a id="Data Preparation">Data Preparation</a>

In [7]:
# Convert 'Date' to datetime & extract 'Month' into own column
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.month

> We extracted `Month` because we took the mean values of `Sunshine` based on month and location from BOM and replaced the null values.

In [8]:
# Drop rows if 'RainTomorrow' is null
df.dropna(subset=['RainTomorrow'], inplace=True)
# Drop 'Evaporation' and 'Date'
df.drop(columns=['Evaporation', 'Date'], inplace=True, axis=1)

> We dropped the rows where `RainTomorrow` was missing because that is our target variable, and we don't want to model on fake data. We dropped `Date` because all we wanted was `Month`. We dropped `Evaporation` because half of the values are missing, and we couldn't find any information to confidently fill them in.

In [9]:
# Change Yes/No to 1/0 for binary classification
df['RainTomorrow'].replace(('Yes', 'No'), (1, 0), inplace=True)
df['RainToday'].replace(('Yes', 'No'), (1, 0), inplace=True)

## <a id="Replacing Sunshine Null Values">Replacing Sunshine Null Values</a>

In [10]:
# Check which locations in 'Sunshine' have missing values
print('Location')
print(df.Location.value_counts())
print()
print(df.groupby(['Location'])['Sunshine'].count())

Location
Canberra            3418
Sydney              3337
Perth               3193
Darwin              3192
Hobart              3188
Brisbane            3161
Adelaide            3090
Bendigo             3034
Townsville          3033
AliceSprings        3031
MountGambier        3030
Launceston          3028
Ballarat            3028
Albany              3016
Albury              3011
PerthAirport        3009
MelbourneAirport    3009
Mildura             3007
SydneyAirport       3005
Nuriootpa           3002
Sale                3000
Watsonia            2999
Tuggeranong         2998
Portland            2996
Woomera             2990
Cairns              2988
Cobar               2988
Wollongong          2983
GoldCoast           2980
WaggaWagga          2976
NorfolkIsland       2964
Penrith             2964
SalmonGums          2955
Newcastle           2955
CoffsHarbour        2953
Witchcliffe         2952
Richmond            2951
Dartmoor            2943
NorahHead           2929
BadgerysCreek   

> Every `Location` has missing values. Some don't even have any values.
>
> We made a nested dictionary of each `Location` with the `Sunshine` mean value for each `Month` taken from <u>BOM</u> to then replace the null values in `Sunshine` using a nested for loop. Three locations (Badgerys Creek, Mount Ginini, and Dartmoor) didn't have `Sunshine` records on <u>BOM</u>, so those values were imputed later.

In [11]:
# Create nested dictionary to replace 'Sunshine' null values with monthly mean recorded values
sunshine = {
    'Adelaide': {1:10.5, 2:10.1, 3:8.6, 4:7.3, 5:5.4, 6:4.5, 7:4.7, 8:6.1, 9:6.8, 10:8.3, 11:9.1, 12:9.5},
    'Albany': {1:8.1, 2:7.4, 3:6.6, 4:6.2, 5:5.4, 6:5.1, 7:5.5, 8:6.1, 9:6.3, 10:6.8, 11:7.4, 12:7.9},
    'Albury': {1:12.1, 2:11.2, 3:9.7, 4: 8.1, 5:5.9, 6:4.9, 7: 4.6, 8:5.9, 9:7.5, 10:9.5, 11:10.9, 12:12},
    'AliceSprings': {1:10.2, 2:9.8, 3:9.7, 4:9.5, 5:8.5, 6:8.4, 7:9.1, 8:9.9, 9:10.0, 10:10.1, 11:10.1, 12:10},
    'Ballarat': {1:10.2, 2:9.3, 3:7.9, 4:6.4, 5:4.6, 6:3.8, 7: 3.9, 8:4.6, 9:5.9, 10:7.2, 11:8.4, 12:9.6},
    'Bendigo': {1:11.9, 2:11.1, 3:9.5, 4:8, 5:5.8, 6:4.9, 7:4.8, 8:5.7, 9:7.2, 10:8.9, 11:10.4, 12:11.6},
    'Brisbane': {1:7.4, 2:6.6, 3:6.5, 4:7.2, 5:6.9, 6:6.8, 7:7.4, 8:7.9, 9:8.2, 10:8.0, 11:8.4, 12:8.1},
    'Cairns': {1:6.8, 2:6.2, 3:6.5, 4:6.8, 5:6.8, 6:7.2, 7:7.4, 8:8.1, 9:8.7, 10:8.8, 11:8.5, 12:7.8},
    'Canberra': {1:9.5, 2:9.0, 3:8.1, 4:7.3, 5:6.0, 6:5.2, 7:5.8, 8:7.0, 9:7.7, 10:8.6, 11:8.9, 12:9.4},
    'Cobar': {1:10.8, 2:10.3, 3:9.6, 4:8.9, 5:7.4, 6:6.4, 7:7.0, 8:8.4, 9:9.1, 10:9.8, 11:10.1, 12:10.6},
    'CoffsHarbour': {1:7.6, 2:7.3, 3:7.1, 4:7.2, 5:6.7, 6:6.6, 7:7.2, 8:8.3, 9:8.5, 10:8.1, 11:7.9, 12:7.9},
    'Darwin': {1:5.7, 2:5.9, 3:6.8, 4:8.8, 5:9.6, 6:10.0, 7:10.2, 8:10.4, 9:9.9, 10:9.5, 11:8.4, 12:6.9},
    'GoldCoast': {1:9.1, 2:8.8, 3:8.4, 4:8.4, 5:8.3, 6:7.8, 7:8.3, 8:8.8, 9:9.2, 10:9.1, 11:9.5, 12:9.6},
    'Hobart': {1:8.2, 2:7.9, 3:6.7, 4:5.9, 5:4.7, 6:4.3, 7:4.8, 8:5.7, 9:6.3, 10:7.4, 11:7.4, 12:7.9},
    'Katherine': {1:8.7, 2:8.6, 3:9.1, 4:9.7, 5:9.9, 6:9.9, 7:10.1, 8:10.4, 9:10.7, 10:11, 11:11, 12:10.1},
    'Launceston': {1:9.6, 2:8, 3:6.9, 4:5.6, 5:4.9, 6:4.3, 7:4.4, 8:4.9, 9:5.9, 10:7, 11:7.9, 12:9.2},
    'Melbourne': {1:9.0, 2:8.1, 3:6.8, 4:5.6, 5:3.9, 6:3.6, 7:3.7, 8:4.7, 9:5.7, 10:6.3, 11:7.0, 12:7.5},
    'MelbourneAirport': {1:8.8, 2:8.1, 3:7.2, 4:6.0, 5:4.7, 6:4.1, 7:4.4, 8:5.4, 9:6.2, 10:7.3, 11:7.5, 12:8.4},
    'Mildura': {1:10.8, 2:10.3, 3:9.6, 4:8.3, 5:6.6, 6:5.5, 7:5.9, 8:7.4, 9:8.2, 10:9.5, 11:10.0, 12:10.7},
    'Moree': {1:10.0, 2:9.8, 3:9.4, 4:9.1, 5:8.1, 6:7.1, 7:7.7, 8:9.0, 9:9.5, 10:9.7, 11:9.6, 12:10.0},
    'MountGambier': {1:9.1, 2:8.6, 3:6.9, 4:5.5, 5:4.4, 6:4.0, 7:4.3, 8:5.3, 9:5.7, 10:7.0, 11:7.5, 12:8.2},
    'Newcastle': {1:9.5, 2:9.2, 3:8.6, 4:8.1, 5:8, 6:7.4, 7:7.8, 8:8.6, 9:9.4, 10:9.4, 11:9.7, 12:10.1},
    'Nhil': {1:11.6, 2:10.8, 3:9.1, 4:7.6, 5:5.7, 6:5.1, 7:5.1, 8:6, 9:7, 10:8.5, 11:10, 12:11},
    'NorfolkIsland': {1:7.5, 2:7.1, 3:6.5, 4:6.5, 5:5.9, 6:5.2, 7:5.9, 8:6.7, 9:7.1, 10:7.5, 11:7.9, 12:7.7},
    'Nuriootpa': {1:10.6, 2:10.0, 3:8.7, 4:7.5, 5:5.7, 6:4.9, 7:5.1, 8:6.5, 9:7.2, 10:8.7, 11:9.6, 12:9.9},
    'PearceRAAF': {1:11.5, 2:10.9, 3:9.5, 4:8.1, 5:6.8, 6:6.0, 7:6.1, 8:7.1, 9:7.8, 10:9.5, 11:10.7, 12:11.5},
    'Penrith': {1:8.1, 2:7.3, 3:7.1, 4:7.4, 5:7.7, 6:7, 7:7.6, 8:8.4, 9:8.9, 10:8.7, 11:8.5, 12:8.8},
    'Perth': {1:10.6, 2:10.0, 3:8.9, 4:7.3, 5:5.9, 6:4.9, 7:5.3, 8:6.2, 9:7.2, 10:8.4, 11:9.8, 12:10.6},
    'PerthAirport': {1:11.5, 2:10.9, 3:9.5, 4:8.1, 5:6.8, 6:6.0, 7:6.1, 8:7.1, 9:7.8, 10:9.5, 11:10.7, 12:11.5},
    'Portland': {1:9.1, 2:8.6, 3:6.9, 4:5.5, 5:4.4, 6:4.0, 7:4.3, 8:5.3, 9:5.7, 10:7.0, 11:7.5, 12:8.2},
    'Richmond': {1:8.1, 2:7.4, 3:7.2, 4:7.3, 5:7.6, 6:6.9, 7:7.6, 8:8.4, 9:8.9, 10:8.7, 11:8.5, 12:8.9},
    'Sale': {1:8.4, 2:7.7, 3:6.8, 4:6.1, 5:4.9, 6:4.4, 7:5.0, 8:5.8, 9:6.4, 10:7.0, 11:7.5, 12:7.9},
    'SalmonGums': {1:9.9, 2:8.8, 3:7.9, 4:7.4, 5:6.4, 6:6.2, 7:5.8, 8:6.5, 9:7.7, 10:8.5, 11:9.5, 12:10.2},
    'Sydney': {1:7.1, 2:6.7, 3:6.4, 4:6.4, 5:5.9, 6:5.5, 7:6.4, 8:7.1, 9:7.2, 10:7.2, 11:7.8, 12:7.6},
    'SydneyAirport': {1:7.5, 2:7.2, 3:6.8, 4:7.0, 5:6.4, 6:5.9, 7:6.7, 8:7.9, 9:8.0, 10:7.9, 11:7.7, 12:7.8},
    'Townsville': {1:8.0, 2:7.4, 3:7.5, 4:7.9, 5:7.5, 6:7.8, 7:8.4, 8:9.1, 9:9.6, 10:9.8, 11:9.5, 12:9.0},
    'Tuggeranong': {1:9.9, 2:8.6, 3:8, 4:7, 5:6.2, 6:5, 7:5.4, 8:6.4, 9:7.7, 10:8.6, 11:9.3, 12:10.2},
    'Uluru': {1:9.2, 2:8.6, 3:8.6, 4:8.1, 5:7.2, 6:6.3, 7:7.3, 8:8, 9:8.6, 10:9.4, 11:9.1, 12:9.4},
    'WaggaWagga': {1:10.8, 2:10.1, 3:9.3, 4:8.2, 5:6.3, 6:4.6, 7:4.8, 8:6.4, 9:7.6, 10:9.2, 11:9.7, 12:10.4},
    'Walpole': {1:8.3, 2:7.1, 3:6.2, 4:4.8, 5:4.5, 6:4.2, 7:4.3, 8:5.0, 9:5.3, 10:6.4, 11:6.5, 12:8.1},
    'Watsonia': {1:7.9, 2:7.9, 3:6.7, 4:5.6, 5:4.0, 6:3.4, 7:3.7, 8:4.9, 9:5.5, 10:6.7, 11:7.2, 12:7.1},
    'Williamtown': {1:7.4, 2:7.2, 3:7.0, 4:6.9, 5:6.1, 6:5.6, 7:6.4, 8:7.5, 9:7.7, 10:7.6, 11:7.6, 12:7.7},
    'Witchcliffe': {1:9.6, 2:9.1, 3:7.9, 4:6.9, 5:6, 6:5.8, 7:5.5, 8:5.8, 9:5.9, 10:6.7, 11:8.6, 12:9.1},
    'Wollongong': {1:8.9, 2:8.3, 3:8, 4:8, 5:8.1, 6:7.4, 7:7.9, 8:8.5, 9:9.1, 10:9.1, 11:9.2, 12:9.5},
    'Woomera': {1:11.0, 2:10.5, 3:9.7, 4:8.8, 5:7.5, 6:6.9, 7:7.3, 8:8.4, 9:9.1, 10:9.7, 11:10.4, 12:10.6}
}
## Sunshine, Funshine~~~

In [12]:
# Replace null 'Sunshine' values with values in {sunshine dictionary}
## Coder's note: I am so proud of these three lines of code!
for loc, sun in sunshine.items():
    for month, hours in sun.items():
        df.loc[((df.Location == loc) & (df.Sunshine.isnull()) & (df.Month == month)), 'Sunshine'] = hours

# Check for any leftover null values in 'Sunshine'
df['Sunshine'].isna().sum()

9206

## <a id="Removing Cloud Cover Values Outside of Range">Removing Cloud Cover Values Outside of Range</a>

In [13]:
# Check value counts of 'Cloud9am' and 'Cloud3pm'
df.Cloud9am.value_counts(), df.Cloud3pm.value_counts()

(7.0    19749
 1.0    15558
 8.0    14389
 0.0     8587
 6.0     8072
 2.0     6442
 3.0     5854
 5.0     5510
 4.0     4373
 9.0        2
 Name: Cloud9am, dtype: int64,
 7.0    18052
 1.0    14827
 8.0    12407
 6.0     8869
 2.0     7153
 3.0     6836
 5.0     6743
 4.0     5254
 0.0     4957
 9.0        1
 Name: Cloud3pm, dtype: int64)

> `Cloud Cover` is measured in oktas (units of eighths) and so a value of 9.0 does not make sense. We don't know the reason for this recording or whether it was misrecorded, so we decided to drop the rows that have a value of 9.0 in both `Cloud9am` and `Cloud3pm` since 3 rows out of 140 is insignificant.

In [14]:
df.drop(((df[df['Cloud9am'] == 9.0]).index & (df[df['Cloud3pm'] == 9.0]).index), inplace=True, axis=0)

## <a id="Removing Outliers">Removing Outliers</a>

In [15]:
# Create a copy to maintain original dataframe
df2 = df.copy()

In [16]:
# Create list of features
features = ['Sunshine', 'MinTemp', 'MaxTemp', 'Rainfall', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm',
            'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Temp9am', 'Temp3pm']


for feature in features:
    q99 = df2[feature].quantile(0.99)
    df2 = df2.loc[(df2[feature].isna()) | (df2[feature] < q99)]

> After parsing through all of the value counts there were some very obviously large outliers, such as days where it rained 371mm compared to the 0-30mm we observed with most the rest of the data. We decided to take out the top 1% of the values on select features.

In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122302 entries, 0 to 145458
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       122302 non-null  object 
 1   MinTemp        121727 non-null  float64
 2   MaxTemp        122011 non-null  float64
 3   Rainfall       121068 non-null  float64
 4   Sunshine       114763 non-null  float64
 5   WindGustDir    113853 non-null  object 
 6   WindGustSpeed  113905 non-null  float64
 7   WindDir9am     113719 non-null  object 
 8   WindDir3pm     118940 non-null  object 
 9   WindSpeed9am   121070 non-null  float64
 10  WindSpeed3pm   119884 non-null  float64
 11  Humidity9am    120747 non-null  float64
 12  Humidity3pm    119203 non-null  float64
 13  Pressure9am    109774 non-null  float64
 14  Pressure3pm    109800 non-null  float64
 15  Cloud9am       75716 non-null   float64
 16  Cloud3pm       72817 non-null   float64
 17  Temp9am        121513 non-nul

> We dropped a lot of data (just over 10%). However, we find this justifiable because the data that was dropped was skewing our data and would make our models more sensitive on the upper end of the data, which with classification models, can lead to more Type I and Type II errors.

## <a id="Preparing Data for Modeling">Preparing Data for Modeling: Imputing and Encoding</a>

In [18]:
# Check for null again
df2.isna().sum()

Location             0
MinTemp            575
MaxTemp            291
Rainfall          1234
Sunshine          7539
WindGustDir       8449
WindGustSpeed     8397
WindDir9am        8583
WindDir3pm        3362
WindSpeed9am      1232
WindSpeed3pm      2418
Humidity9am       1555
Humidity3pm       3099
Pressure9am      12528
Pressure3pm      12502
Cloud9am         46586
Cloud3pm         49485
Temp9am            789
Temp3pm           2310
RainToday         1234
RainTomorrow         0
Month                0
dtype: int64

In [19]:
# Set target variable and features variable
X = df2.drop('RainTomorrow', axis=1)
y = df2['RainTomorrow']

# Split data in training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Export y_train and y_test to use in modeling notebook
    #y_train.to_parquet("y_train.parquet")
    #y_test.to_parquet("y_test.parquet")

In [20]:
# Use SimpleImputer
cols_simp = ['WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday']
simputer = SimpleImputer(strategy = 'most_frequent')
simputed = simputer.fit_transform(X_train[cols_simp])
df_simp_transformed = pd.DataFrame(simputed, index=X_train.index, columns = cols_simp)

> Because these columns did not have large amounts of null data we decided to replace the null values with the most frequent value, using a SimpleImputer, in each of these features.

In [21]:
X_train_1 = X_train.copy()
X_train_1.drop(columns = cols_simp, axis=1, inplace=True)
X_train_imp = pd.concat((X_train_1, df_simp_transformed), axis=1)

> We dropped old non-imputed features and created a copy of the old dataframe, which we then concatenated to the new imputed columns.

In [22]:
# Use OneHotEncoder for categorical features
cols_encode = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'Month']
ohe = OneHotEncoder(sparse = False, handle_unknown='ignore')
ohe_fit = ohe.fit_transform(X_train_imp[cols_encode])
ohe_df = pd.DataFrame(ohe_fit, columns = ohe.get_feature_names(cols_encode), index = X_train_imp.index)

In [23]:
X_train_int = X_train_imp.drop(cols_encode, axis=1)
X_train_ohe = pd.concat((X_train_int, ohe_df), axis=1)

> Same process as with SimpleImputer. We narrowed down to the relevant categorical features in our dataset and applied OneHotEncoder. We then dropped the unencoded columns from the dataframe and concatenated the encoded columns to our updated dataframe. 

In [24]:
X_train_ohe.head()

,MinTemp,MaxTemp,Rainfall,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,...,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
20387,13.8,18.5,0.0,NaN,54.0,11.0,19.0,75.0,84.0,1022.9,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
32950,14.6,23.6,0.0,9.3,28.0,6.0,19.0,53.0,44.0,1026.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
46812,16.6,20.6,4.8,0.9,28.0,6.0,7.0,95.0,89.0,1014.1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44990,16.9,21.2,11.4,9.2,44.0,9.0,26.0,89.0,93.0,1012.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
112183,9.5,17.5,2.4,5.5,48.0,9.0,9.0,99.0,77.0,1018.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [25]:
# Implement KNNImputer to impute rest of missing values
imputer = KNNImputer(n_neighbors=5, weights="distance")
imputer_train = imputer.fit_transform(X_train_ohe)
X_train_KNNI = pd.DataFrame(imputer_train, index = X_train_ohe.index, columns = X_train_ohe.columns)
X_train_KNNI.head()

,MinTemp,MaxTemp,Rainfall,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,...,Month_3,Month_4,Month_5,Month_6,Month_7,Month_8,Month_9,Month_10,Month_11,Month_12
20387,13.8,18.5,0.0,8.402434,54.0,11.0,19.0,75.0,84.0,1022.9,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
32950,14.6,23.6,0.0,9.300000,28.0,6.0,19.0,53.0,44.0,1026.3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
46812,16.6,20.6,4.8,0.900000,28.0,6.0,7.0,95.0,89.0,1014.1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44990,16.9,21.2,11.4,9.200000,44.0,9.0,26.0,89.0,93.0,1012.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
112183,9.5,17.5,2.4,5.500000,48.0,9.0,9.0,99.0,77.0,1018.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# Final null check on X_train_KNNI
X_train_KNNI.isna().sum().sum()

0

> We repeat this process for our test data.

In [27]:
# SimpleImputer on X_test
simp_test = simputer.transform(X_test[cols_simp])
df_simp_test = pd.DataFrame(simp_test, index=X_test.index, columns=cols_simp)
X_test_1 = X_test.copy()
X_test_1.drop(columns=cols_simp, axis=1, inplace=True)
X_test_imp = pd.concat((X_test_1, df_simp_test), axis=1)

In [28]:
# OHE on X_test_imp
ohe_test = ohe.transform(X_test_imp[cols_encode])
df_ohe_test = pd.DataFrame(ohe_test, columns=ohe.get_feature_names(cols_encode), index=X_test_imp.index)
X_test_2 = X_test_imp.copy()
X_test_2.drop(columns=cols_encode, axis=1, inplace=True)
X_test_ohe = pd.concat((X_test_2, df_ohe_test), axis=1)

In [29]:
# KNNImputer on X_test_ohe
imputer_test = imputer.transform(X_test_ohe)
X_test_KNNI = pd.DataFrame(imputer_test, index=X_test_ohe.index, columns=X_test_ohe.columns)

In [30]:
# Final null check on X_test_KNNI
X_test_KNNI.isna().sum().sum()

0

> Nice! Now all of our null values are taken care of. However, we still observed some "impossible" values when we imputed values for `Cloud Cover`. We know from our research that the units for this are in oktas, which can only range from 1-8 as whole numbers. To achieve this, we rounded to the nearest whole number.

In [36]:
X_train_KNNI['Cloud9am'] = round(X_train_KNNI['Cloud9am'])
X_train_KNNI['Cloud3pm'] = round(X_train_KNNI['Cloud3pm'])

X_test_KNNI['Cloud9am'] = round(X_test_KNNI['Cloud9am'])
X_test_KNNI['Cloud3pm'] = round(X_test_KNNI['Cloud3pm'])

> Finally, we saved our training and test data to a `.parquet` file so that we don't have to run a computationally expensive (and slow) imputer every single time we come back to this notebook or when running our modeling notebook.

In [37]:
# Export X_train_KNNI and X_test_KNNI as .parquet
    # X_train_KNNI.to_parquet("X_train_KNNI.parquet")
    # X_test_KNNI.to_parquet("X_test_KNNI.parquet")

---

# <a id="Final Remarks">Final Remarks</a>

> Given the large size of our dataset, we felt confident in imputing null values. However, we are aware of the limitations that this might add to our dataset given the amount of null values that have been synthetically created. Further investigations into obtaining actual data would be warranted with additional resources and time.
>
> Refer to our [modeling notebook](https://github.com/aykim1127/project3/blob/main/modeling_notebook.ipynb) for our iterative modeling process and final results.